In [1]:
import numpy as np
import pandas as pd

from skimage import io
from skimage.transform import resize
from skimage.feature import graycomatrix, graycoprops
from skimage.color import rgb2gray
from skimage import img_as_ubyte, img_as_float64

In [2]:
folders = ['Agua/*.*', 'Bosque/*.*', 'Ciudad/*.*', 'Cultivo/*.*', 'Desierto/*.*', 'Montaña/*.*']
nbins = 16

# features de imagenes

In [ ]:
imagenes = pd.DataFrame()

In [47]:
id = 0
for folder in folders:
    raw_folder = io.imread_collection(folder)
    for img in raw_folder:
        resized_img = resize(img, (128,128), anti_aliasing=True)

        rh = np.histogram(resized_img[:,:,0].flatten(), nbins, density = True)
        gh = np.histogram(resized_img[:,:,1].flatten(), nbins, density = True)
        bh = np.histogram(resized_img[:,:,2].flatten(), nbins, density = True)
        hist_descriptor = np.concatenate((rh[0], gh[0], bh[0]))

        gray_resized = img_as_ubyte(rgb2gray(resized_img))
        glcm = graycomatrix(gray_resized, distances=[5], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4])
        texture_desc = [graycoprops(glcm, 'dissimilarity')[0, 0], graycoprops(glcm, 'homogeneity')[0, 0], graycoprops(glcm, 'energy')[0, 0], graycoprops(glcm, 'correlation')[0, 0]]

        img_descr = np.concatenate(([id], hist_descriptor, texture_desc))

        img_df = pd.DataFrame([img_descr])
    
        imagenes = pd.concat([imagenes, img_df])
    id += 1

# imagenes a texto

In [3]:
imagenes_color = pd.DataFrame()
imagenes_gris = pd.DataFrame()

In [6]:
id = 0
for folder in folders:
    raw_folder = io.imread_collection(folder)
    for img in raw_folder:
        resized_img = resize(img, (128,128), anti_aliasing=True)

        gray_resized = img_as_float64(rgb2gray(resized_img))

        img_color = np.concatenate(([id], img_as_float64(resized_img).flatten()))
        img_grises = np.concatenate(([id], gray_resized.flatten()))

        color_df = pd.DataFrame([img_color])
        grises_df = pd.DataFrame([img_grises])
    
        imagenes_color = pd.concat([imagenes_color, color_df])
        imagenes_gris = pd.concat([imagenes_gris, grises_df])
    id += 1

In [8]:
imagenes_color.shape

(2016, 49153)

In [9]:
imagenes_color.to_csv('satelites_color.csv', index=False, header=False)
imagenes_gris.to_csv('satelites_grises.csv', index=False, header=False)

In [5]:
len(img_as_ubyte(io.imread('testing.jpg')).flatten())

6220800